# Algoritmos de optimización - Reto 3
Descripcion: Implementar el algoritmo de búsqueda tabú para el TSP de la AG3 <br>
Nombre: José Manuel Perez Utrilla  <br>
Github: <br/3.

In [ ]:
!pip install requests


!pip install tsplib95


In [2]:
import urllib.request
import tsplib95
import random   #Libreria para generar numeros y listas aleatorias
import copy     #Permite hacer copias de objetos en python: listas, diccionarios,...
import gzip
import shutil
import numpy as np

In [6]:
#Librerias y carga del problema

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/usage.html
  # https://tsplib95.readthedocs.io/modules.html#module-tsplib95.models

#Descargamos el fichero de datos(Matriz de distancias)
#file = "swiss42.tsp" ;
#urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
#!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

# Descargar el fichero de datos
file = "swiss42.tsp"
gz_file = file + ".gz"
url = "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz"

# Descargar el archivo comprimido
urllib.request.urlretrieve(url, gz_file)

# Descomprimir el archivo usando gzip
with gzip.open(gz_file, 'rb') as f_in:
    with open(file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        


In [8]:
#Objeto de tsplib95 para nuestro problema problema
problem = tsplib95.load_problem(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

#Coordenadas(si estan disponibles en el ficher)
problem.get_display(1)

#Distancia
problem.get_weight(1, 2)

#Matriz de distancias
[ [problem.get_weight(i, j)   for i in  list(problem.get_nodes()) ]  for j in  list(problem.get_nodes()) ]

C:\Users\Jose\AppData\Local\Temp\ipykernel_40364\3396158449.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) load_problem. (Will be removed in newer versions. Use `tsplib95.load` instead.) -- Deprecated since version 7.0.0.
  problem = tsplib95.load_problem(file)


[[0,
  15,
  30,
  23,
  32,
  55,
  33,
  37,
  92,
  114,
  92,
  110,
  96,
  90,
  74,
  76,
  82,
  67,
  72,
  78,
  82,
  159,
  122,
  131,
  206,
  112,
  57,
  28,
  43,
  70,
  65,
  66,
  37,
  103,
  84,
  125,
  129,
  72,
  126,
  141,
  183,
  124],
 [15,
  0,
  34,
  23,
  27,
  40,
  19,
  32,
  93,
  117,
  88,
  100,
  87,
  75,
  63,
  67,
  71,
  69,
  62,
  63,
  96,
  164,
  132,
  131,
  212,
  106,
  44,
  33,
  51,
  77,
  75,
  72,
  52,
  118,
  99,
  132,
  132,
  67,
  139,
  148,
  186,
  122],
 [30,
  34,
  0,
  11,
  18,
  57,
  36,
  65,
  62,
  84,
  64,
  89,
  76,
  93,
  95,
  100,
  104,
  98,
  57,
  88,
  99,
  130,
  100,
  101,
  179,
  86,
  51,
  4,
  18,
  43,
  45,
  95,
  45,
  115,
  93,
  152,
  159,
  100,
  112,
  114,
  153,
  94],
 [23,
  23,
  11,
  0,
  11,
  48,
  26,
  54,
  70,
  94,
  69,
  89,
  75,
  84,
  84,
  89,
  92,
  89,
  54,
  78,
  99,
  141,
  111,
  109,
  190,
  89,
  44,
  11,
  29,
  54,
  56,
  89,
  47,
  1

In [16]:
#Funciones de la Actividad Guiada 3
matriz_distancias = np.zeros((len(Nodos), len(Nodos)))

for i in range(len(Nodos)):
    for j in range(len(Nodos)):
        matriz_distancias[i][j] = problem.get_weight(Nodos[i], Nodos[j])
        
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:] :
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)


#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [22]:
# Implementación de la Búsqueda Tabú

def busqueda_tabu(problem, tamano_lista_tabu, num_iteraciones):
    """
    Implementa el algoritmo de Búsqueda Tabú para resolver el Problema del Viajante (TSP).
    
    La Búsqueda Tabú es una metaheurística que explora el espacio de soluciones 
    intercambiando nodos en la ruta para encontrar una mejor solución sin quedar atrapada 
    en óptimos locales, utilizando una lista tabú para evitar ciclos repetitivos.
    
    :param problem: Instancia del problema de TSPLIB95
    :param tamano_lista_tabu: Tamaño máximo de la lista tabú, que restringe ciertos movimientos
    :param num_iteraciones: Número de iteraciones en la búsqueda de soluciones
    :return: La mejor solución encontrada y su distancia total
    """
    # Convertimos explícitamente los nodos en una lista para evitar errores con iteradores
    Nodos = list(problem.get_nodes())
    num_ciudades = len(Nodos)
    
    # Se genera una solución inicial aleatoria
    solucion_optima = crear_solucion(Nodos)
    distancia_optima = distancia_total(solucion_optima, problem)
    solucion_actual = solucion_optima.copy()
    lista_tabu = []
    
    for iteracion in range(num_iteraciones):
        movimiento_encontrado = False
        mejor_movimiento = None
        mejor_distancia_movimiento = float('inf')
        
        # Se generan soluciones vecinas intercambiando dos nodos en la ruta
        for i in range(1, num_ciudades - 1):
            for j in range(i + 1, num_ciudades):
                # Crear una nueva solución modificando la actual
                movimiento = solucion_actual.copy()
                movimiento[i], movimiento[j] = movimiento[j], movimiento[i]
                distancia_movimiento = distancia_total(movimiento, problem)
                
                # Se verifica si la nueva solución es mejor y no está en la lista tabú
                if distancia_movimiento < mejor_distancia_movimiento and (i, j) not in lista_tabu:
                    mejor_movimiento = movimiento 
                    mejor_distancia_movimiento = distancia_movimiento 
                    movimiento_encontrado = True

        # Si se encontró un movimiento mejor, se actualiza la solución
        if movimiento_encontrado:
            solucion_actual = mejor_movimiento
            lista_tabu.append((i, j))
            
            # Se mantiene el tamaño de la lista tabú dentro del límite
            if len(lista_tabu) > tamano_lista_tabu:
                lista_tabu.pop(0)

            # Se actualiza la mejor solución si se encuentra una mejor
            if mejor_distancia_movimiento < distancia_optima:
                solucion_optima = mejor_movimiento
                distancia_optima = mejor_distancia_movimiento
    
    return solucion_optima, distancia_optima

# Ejecutar la búsqueda tabú con parámetros definidos
solucion_optima, distancia_optima = busqueda_tabu(problem, tamano_lista_tabu=5, num_iteraciones=1000)

# Imprimir la mejor solución encontrada y su distancia
print("Solución óptima encontrada:", solucion_optima)
print("Distancia óptima encontrada:", distancia_optima)



Solución óptima encontrada: [0, 38, 22, 13, 19, 14, 16, 15, 37, 17, 31, 7, 1, 6, 5, 4, 3, 32, 34, 33, 20, 35, 36, 27, 2, 28, 30, 29, 8, 9, 39, 24, 40, 21, 23, 41, 10, 25, 11, 12, 18, 26]
Distancia óptima encontrada: 1712


In [24]:
solucion_optima, distancia_optima = busqueda_tabu(problem, tamano_lista_tabu=10, num_iteraciones=1500)

# Imprimir resultados
print("Solución óptima encontrada:", solucion_optima)
print("Distancia óptima encontrada:", distancia_optima)

Solución óptima encontrada: [0, 3, 1, 31, 20, 33, 34, 30, 32, 7, 15, 16, 14, 6, 4, 29, 24, 40, 23, 41, 25, 10, 2, 27, 28, 38, 22, 39, 21, 9, 8, 26, 5, 18, 12, 11, 13, 19, 37, 36, 35, 17]
Distancia óptima encontrada: 1902
